# Refactor Benchmark OPF

This notebook aims to beckmark the optimal power flow.

In [1]:
import warnings
import logging
import ams
import numpy as np
import pandas as pd

import ams.benchmarks as bp

# Configure AMS logger
ams.config_logger(stream_level=20)

# Display tool versions
_ = bp.get_tool_versions()

ams.config_logger(stream_level=50)

Last run time: 2024-11-05 19:01:47
Python: 3.10.0 | packaged by conda-forge | (default, Nov 20 2021, 02:27:15) [Clang 11.1.0 ]

Tool        Version                
----------  -----------------------
ltbams      0.9.10.post18+g43b7dbe3
cvxpy       1.5.3                  
pandapower  2.14.11                
PYPOWER     5.1.17                 
gurobipy    11.0.3                 
mosek       10.2.6                 
ecos        2.0.14                 
scs         3.2.7                  
piqp        0.4.2                  
numba       0.60.0                 


In [2]:
%matplotlib inline

In [3]:
cases = [
    './cases/case14.m',
    './cases/case39.m',
    './cases/case89pegase.m',
    './cases/case118.m',
    './cases/npcc.m',
    './cases/wecc.m',
    './cases/case300.m',
    './cases/pglib_opf_case1354_pegase.m',
    './cases/pglib_opf_case2869_pegase.m',
    './cases/pglib_opf_case4020_goc.m',
    './cases/pglib_opf_case5658_epigrids.m',
    './cases/pglib_opf_case7336_epigrids.m',
]

In [4]:
# run a single case to ensure everything is working
_t, _obj = bp.test_time(cases[0], routine='DCOPF', ignore_dpp=True)
print(f"Test run successful.\nTime: {_t}\nObjective: {_obj}")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-25


/Users/jinningwang/work/miniconda3/envs/ams/lib/python3.10/site-packages/pandapower/create.py:5049: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  net[table] = pd.concat([net[table], dd[dd.columns[~dd.isnull().all()]]], sort=False)
There are 3 branches which are considered as trafos - due to ratio unequal 0 or 1 - but connect same voltage levels.
gen vm_pu > bus max_vm_pu for gens [2 3]. Setting bus limit for these gens.


Test run successful.
Time: [0.0009, 0.0033, 0.0029, 0.0, 0.0001, 0.0131, 0.0177, 0.0043, 0.1316]
Objective: [7642.591781969892, 7642.591729476089, 7642.591777143645, 7642.591776991807]


In [5]:
# Suppress warnings
warnings.filterwarnings('ignore')
# show only errors
logging.getLogger('pandapower').setLevel(logging.ERROR)

# Initialize data structures for storing results
n_iters = 10
n_cases = len(cases)

time_data = np.zeros((n_iters, n_cases, len(bp.cols_time)))
obj_data = np.zeros((n_cases, len(bp.cols_obj)))

In [6]:
# Run the benchmark for each case and iteration
for n_case, case in enumerate(cases):
    print(f'Case: {case}')
    for n_iter in range(n_iters):
        time, obj = bp.test_time(case)
        time_data[n_iter, n_case, :] = time
    obj_data[n_case, :] = obj

# Optionally, re-enable warnings if needed elsewhere in your code
warnings.filterwarnings('default')

Case: ./cases/case14.m
Case: ./cases/case39.m
Case: ./cases/case89pegase.m
Case: ./cases/case118.m
Case: ./cases/npcc.m
Case: ./cases/wecc.m
Case: ./cases/case300.m
Case: ./cases/pglib_opf_case1354_pegase.m
Case: ./cases/pglib_opf_case2869_pegase.m
Case: ./cases/pglib_opf_case4020_goc.m
Case: ./cases/pglib_opf_case5658_epigrids.m
Case: ./cases/pglib_opf_case7336_epigrids.m


In [7]:
# Process and display the results
case_names = [case.split('/')[-1].split('.')[0] for case in cases]

obj_dcopf = pd.DataFrame(obj_data, columns=bp.cols_obj, index=case_names)

time_dcopf = pd.DataFrame(columns=bp.cols_time, index=case_names)
for case in case_names:
    time_dcopf.loc[case] = time_data[:, case_names.index(case), :].mean(axis=0)

time_dcopf.iloc[:, :] *= 1000  # Convert time to milliseconds

In [8]:
time_dcopf

,ams_mats,ams_parse,ams_eval,ams_final,ams_postinit,ams_grb,ams_mosek,ams_piqp,pdp
case14,0.72,1.58,2.28,0.0,0.1,5.67,6.03,4.06,14.17
case39,0.71,1.53,2.33,0.0,0.1,6.18,6.4,4.28,14.57
case89pegase,1.31,1.59,2.42,0.0,0.1,14.73,8.39,4.98,20.07
case118,1.3,2.39,5.76,0.0,0.1,15.29,10.14,4.97,27.65
npcc,1.41,2.2,4.9,0.0,0.1,9.81,10.96,5.19,65.4
wecc,1.5,1.8,3.41,0.0,0.1,9.54,15.54,5.36,39.66
case300,2.14,2.95,7.8,0.0,0.1,27.33,15.86,6.59,37.67
pglib_opf_case1354_pegase,9.12,18.11,67.67,0.02,0.1,53.65,42.54,17.65,541.13
pglib_opf_case2869_pegase,19.84,63.55,246.32,0.18,0.1,133.67,104.54,39.87,1485.74
pglib_opf_case4020_goc,25.74,33.33,126.16,0.12,0.1,300.03,191.07,147.27,2434.02


In [9]:
obj_dcopf.round(2)

,grb,mosek,piqp,pdp
case14,7642.59,7642.59,7642.59,7642.59
case39,41263.94,41263.94,41263.94,41263.94
case89pegase,5733.37,5733.37,5733.37,5733.37
case118,125947.88,125947.88,125947.88,125947.88
npcc,810033.37,810033.37,810033.37,810016.06
wecc,411706.13,411706.13,411706.13,411706.13
case300,706292.32,706292.32,706292.32,706292.32
pglib_opf_case1354_pegase,1218096.86,1218096.86,1218096.86,1218096.86
pglib_opf_case2869_pegase,2386235.33,2386235.32,2386235.33,2386235.33
pglib_opf_case4020_goc,793634.11,793634.11,793634.11,793634.11


In [10]:
# export results to csv
obj_dcopf.to_csv('../results_obj.csv')
time_dcopf.to_csv('../results_time.csv')